# 000 Forecasting Bot

Starting from https://colab.research.google.com/drive/1_Il5h2Ed4zFa6Z3bROVCE68LZcSi4wHX?usp=sharing

## Imports

In [1]:
from IPython.display import Markdown
from omegaconf import OmegaConf
import datetime,json, os, requests, re
from openai import OpenAI
from tqdm import tqdm

## Today

In [2]:
import datetime
today = str(datetime.datetime.now())[0:10]
today

'2024-08-03'

## API Keys

In order to run this notebook as is, you'll need to enter a few API keys (use the key icon on the left to input them):

- `METACULUS_TOKEN`: you can find your Metaculus token under your bot's user settings page: https://www.metaculus.com/accounts/settings/, or on the bot registration page where you created the account: https://www.metaculus.com/aib/
- `OPENAPI_API_KEY`: get one from OpenAIs page: https://platform.openai.com/settings/profile?tab=api-keys
- `PERPLEXITY_API_KEY` - used to search up-to-date information about the question. Get one from https://www.perplexity.ai/settings/api
- `ASKNEWS_CLIENT_ID`, `ASKNEWS_SECRET`

In [3]:
token_fn = "tokens.yaml"

In [4]:
config = OmegaConf.load(token_fn)

def pr(tokens):
    print(OmegaConf.to_yaml(config))

## LLM and Metaculus Interaction

This section sets up some simple helper code you can use to get data about forecasting questions and to submit a prediction

In [5]:
AUTH_HEADERS = {"headers": {"Authorization": f"Token {config.METACULUS_TOKEN}"}}
API_BASE_URL = "https://www.metaculus.com/api2"
WARMUP_TOURNAMENT_ID = 3349
SUBMIT_PREDICTION = True

def find_number_before_percent(s):
    # Use a regular expression to find all numbers followed by a '%'
    matches = re.findall(r'(\d+)%', s)
    if matches:
        # Return the last number found before a '%'
        return int(matches[-1])
    else:
        # Return None if no number found
        return None

def post_question_comment(question_id, comment_text):
    """
    Post a comment on the question page as the bot user.
    """

    response = requests.post(
        f"{API_BASE_URL}/comments/",
        json={
            "comment_text": comment_text,
            "submit_type": "N",
            "include_latest_prediction": True,
            "question": question_id,
        },
        **AUTH_HEADERS,
    )
    response.raise_for_status()
    print("Comment posted for ", question_id)

def post_question_prediction(question_id, prediction_percentage):
    """
    Post a prediction value (between 1 and 100) on the question.
    """
    url = f"{API_BASE_URL}/questions/{question_id}/predict/"
    response = requests.post(
        url,
        json={"prediction": float(prediction_percentage) / 100},
        **AUTH_HEADERS,
    )
    response.raise_for_status()
    print("Prediction posted for ", question_id)


def get_question_details(question_id):
    """
    Get all details about a specific question.
    """
    url = f"{API_BASE_URL}/questions/{question_id}/"
    response = requests.get(
        url,
        **AUTH_HEADERS,
    )
    response.raise_for_status()
    return json.loads(response.content)

def list_questions(tournament_id=WARMUP_TOURNAMENT_ID, offset=0, count=1000):
    """
    List (all details) {count} questions from the {tournament_id}
    """
    url_qparams = {
        "limit": count,
        "offset": offset,
        "has_group": "false",
        "order_by": "-activity",
        "forecast_type": "binary",
        "project": tournament_id,
        "status": "open",
        "type": "forecast",
        "include_description": "true",
    }
    url = f"{API_BASE_URL}/questions/"
    response = requests.get(url, **AUTH_HEADERS, params=url_qparams)
    response.raise_for_status()
    data = json.loads(response.content)
    return data

## IFP

In [95]:
class IFP:

    forecast_fields = ['question_id',
                       'title',
                       'feedback',
                       'resolution_criteria', 
                       'background', 
                       'fine_print',
                       'event', 
                       'model_domain',
                       'news',
                       'rates', 
                       'research']

    forecast_format = f"|{'|'.join(forecast_fields)}|"

    def record(self):
        rec = vars(self)
        sep = '|'.join([f"{x}: {rec[x]}" for x in self.forecast_fields])
        return f'|{sep}|'

    def __init__(self, question_id):
        self.question_id = question_id
        self.question_details = get_question_details(self.question_id)
        self.today = datetime.datetime.now().strftime("%Y-%m-%d")   
        self.title = self.question_details["title"]
        self.resolution_criteria = self.question_details["resolution_criteria"]
        self.background = self.question_details["description"]
        self.fine_print = self.question_details["fine_print"]
        self.event = ''
        self.model_domain = ''
        self.news = ''
        self.rates = ''
        self.research = ''
        self.feedback = ''

    def report(self):
        rpt = f"""
The future event is described by this question: [ {self.title} ]
The resolution criteria are: [ {self.resolution_criteria} ]
The background is: [ {self.background} ]"""
        if self.fine_print:
            rpt += f"""
The fine print is: [ {self.fine_print} ]"""
        return rpt

    def upload(self):
        post_question_prediction(self.question_id, self.forecast)
        post_question_comment(self.question_id, self.rationale)

In [7]:
unit_test = True

In [8]:
if unit_test:
    qid = 26775
    ifp = IFP(qid)
    ifps = {qid: ifp}
    print(ifp.record())

|question_id: 26775|title: Will someone other than Fabiano Caruana, Alireza Firouzja, Praggnanandhaa Rameshbabu, or Gukesh Dommaraju win the 2024 Grand Chess Tour?|feedback: |resolution_criteria: This question will resolve Yes if someone other than Fabiano Caruana, Alireza Firouzja, Praggnanandhaa Rameshbabu, or Gukesh Dommaraju wins the 2024 Grand Chess Tour, according to the Grand Chess Tour site or credible sources.|background: The Grand Chess Tour consists of [five chess tournaments](https://grandchesstour.org/tours/2024/), where the nine Tour players (listed above) can recieve Grand Chess Tour points, according to their place in the tournaments. Three of the five tournaments are already finished with the "2024 SAINT LOUIS RAPID & BLITZ" and the "2024 SINQUEFIELD CUP" still to go.  Leaving one tournament with the quicker time controls of "Blitz" and "Rapid" as well as one tournament in a longer, "Classical" time control still to be played.

The [current standings](https://grandches

In [9]:
IFP.forecast_format

'|question_id|title|feedback|resolution_criteria|background|fine_print|event|model_domain|news|rates|research|'

## News source (AskNews)

https://colab.research.google.com/drive/1tc383HraMZOiyfKFF1EXAtlTYbsuv3Q5?usp=sharing

In [119]:
from asknews_sdk import AskNewsSDK

def asknews_to_dict(x):
    return dict([x.split(': ', 1) for x in x.split('\n')])

class AskNews():

    def __init__(self):
        ASKNEWS_CLIENT_ID = config.ASKNEWS_CLIENT_ID
        ASKNEWS_SECRET = config.ASKNEWS_SECRET
        
        self.ask = AskNewsSDK(
              client_id=config.ASKNEWS_CLIENT_ID,
              client_secret=config.ASKNEWS_SECRET,
              scopes=["news"]
          )
        
    def query(self, 
              q, 
              method,    # use "nl" for natural language for your search, or "kw" for keyword, or 'both'
              strategy): # strategy="latest news" enforces looking at the latest news only
                         # strategy="news knowledge" looks for relevant news within the past 60 days
        return ask.ask.news.search_news(
            query=q, # your keyword query
            n_articles=10, # control the number of articles to include in the context
            return_type="dicts",  # you can also ask for "dicts" if you want more information
            method=method, 
            strategy=strategy).as_dicts # strategy="latest news" enforces looking at the latest news only,
                               
    def research(self, ifp):
        q = ifp.title.strip()
        all = []
        for strategy in ['latest news', 'news knowledge']:
            for x in self.query(q, 'both', strategy):
                all.append(x.summary)
        all = list(set(all))
        ifp.news = '\n\n'.join(all)
        print('News for', ifp.question_id)
        print(ifp.news)
        return ifp.news

In [11]:
if unit_test:
    ask = AskNews()
    news = ask.research(ifp)

The Superbet Chess Classic Romania resumed after a rest day, with GM Fabiano Caruana maintaining his lead after drawing with GM Ian Nepomniachtchi. GM Alireza Firouzja joined GMs Praggnanandhaa Rameshbabu and Gukesh Dommaraju in second place after defeating GM Wesley So. The games of GM Nodirbek Abdusattorov vs. GM Anish Giri and GM Maxime Vachier-Lagrave vs. GM Gukesh Dommaraju also ended in draws. The tournament will continue with the seventh round on July 3.

World Chess Champion Magnus Carlsen lost to Richard Rapport of Hungary in the 4th round of the World Rapid and Blitz Chess Team Championship. Carlsen, playing for the WR Chess Team, lost in 23 moves with white pieces. His teammate, Russian grandmaster Ian Nepomniachtchi, drew with Hikaru Nakamura's opponent, Aronian's teammate, in his match. As a result, WR Chess Team lost 1-5 to Chessy. After five rounds, Al-Ain ACMG UAE is leading with 21.5 points, with Chessy trailing by 0.5 points. WR Chess Team, with Carlsen and Nepomniach

## LLMs

### In general

In [12]:
class LLM:
    def __init__(self, system_role):
        self.system_role = system_role
        self.messages = [{"role": "system", "content": system_role}]

    def chat(self, query):
        self.messages.append({"role": "user", "content": query})
        text = self.message()
        self.messages.append({"role": "assistant", "content": text})
        return text

### MetaAI

https://pypi.org/project/meta-ai-api/1.0.6/

In [13]:
from meta_ai_api import MetaAI as mai

In [73]:
class MetaAI(LLM):

    def __init__(self, system_role = "Nice talker"):
        self.ai = mai()
        super().__init__(system_role)
        
    def message(self):
        self.response = self.ai.prompt(message=self.messages[-1]['content'])
        return self.response['message']

In [74]:
if unit_test:
    ai = MetaAI()
    r = ai.chat(ifp.title)
    print(r)

Will someone other than Fabiano Caruana, Alireza Firouzja, Praggnanandhaa Rameshbabu, or Gukesh Dommaraju win the 2024 Grand Chess Tour?
The 2024 Grand Chess Tour is still ongoing, and the current standings are as follows ¹:
Fabiano Caruana: 22.25 points
Alireza Firouzja: 17.58 points
Praggnanandhaa Rameshbabu: 16.25 points
Gukesh D: 14.25 points
Other players, such as Magnus Carlsen, Wei Yi, and Jan-Krzysztof Duda, have also performed well in individual tournaments ². However, it's important to note that the tour consists of multiple events, and the overall winner will be determined by the cumulative points earned throughout the tour.
The tour's format and scoring system are as follows ¹:
Two classical tournaments with a time control of 25 minutes plus a 10-second increment per move
Three rapid and blitz tournaments with a time control of 5+2
Tour points are awarded based on the combined results of both portions
The player with the most tour points at the end of the five tournaments w

### HuggingChat

https://pypi.org/project/hugchat/

In [16]:
from hugchat import hugchat
from hugchat.login import Login

In [80]:
class HuggingChat(LLM):

    def __init__(self, system_role = "Nice talker"):
        # Log in to huggingface and grant authorization to huggingchat
        EMAIL = config.HUGGINGFACE_USERNAME
        PASSWD = config.HUGGINGFACE_PASSWORD
        cookie_path_dir = "./cookies/" # NOTE: trailing slash (/) is required to avoid errors
        sign = Login(EMAIL, PASSWD)
        cookies = sign.login(cookie_dir_path=cookie_path_dir, save_cookies=True)
        self.chatbot = hugchat.ChatBot(cookies=cookies.get_dict()) 
        super().__init__(system_role)
        
    def message(self):
        result = ''.join([x['token'] for x in self.chatbot.chat(self.system_role + '\n\n' + self.messages[-1]['content']) if x])
        return result

    def web_search(self, query):
        query_result = self.chatbot.query(query, web_search=True)
        return query_result
        print(query_result)
        for source in query_result.web_search_sources:
            print(source.link)
            print(source.title)
            print(source.hostname)

    def available_models(self):
        return [(i,str(x)) for i,x in enumerate(self.chatbot.get_available_llm_models())]

    def switch_llm(self, i):
        self.chatbot.switch_llm(i)

    def conversation_info(self):
        info = self.chatbot.get_conversation_info()
        # print(info.id, info.title, info.model, info.system_prompt, info.history)
        return info

In [81]:
if unit_test:
    hc = HuggingChat()
    hc.switch_llm(1)
    print(hc.chat(ifp.title))

It's difficult to predict with certainty, but I can provide some insights.

The 2024 Grand Chess Tour will likely feature a strong field of top grandmasters, and while Fabiano Caruana, Alireza Firouzja, Praggnanandhaa Rameshbabu, and Gukesh Dommaraju are indeed among the favorites, there are other talented players who could potentially challenge for the title.

Some possible dark horses to consider:

1. Maxime Vachier-Lagrave (France): A consistent top-10 player, Vachier-Lagrave has a strong all-around game and has shown the ability to perform well in rapid and blitz formats.
2. Viswanathan Anand (India): The former World Champion has been playing some of the best chess of his life in recent years, and his experience and skill make him a formidable opponent.
3. Sergey Karjakin (Russia): A former World Championship challenger, Karjakin has a strong record in rapid and blitz events and is always a threat to win.
4. Leinier Domínguez (USA): A versatile player with a strong tactical sense,

### Metaculus Anthropic

In [82]:
import json

class Claude(LLM):
       
    def message(self):
        content = self.messages[-1]['content']
        url = "https://www.metaculus.com/proxy/anthropic/v1/messages"
        headers =  {"Authorization": f"Token {config.METACULUS_TOKEN}",
                    "anthropic-version": "2023-06-01",
                    "content-type": "application/json"}
        data = {"model": "claude-3-5-sonnet-20240620",
                "max_tokens": 1024,
                "messages": [{"role": "user", "content": self.system_role + '\n\n' + content}]}
        response = requests.post(url, headers=headers, data = json.dumps(data))
        # Check the response status code
        if response.status_code != 200:
            print("Request failed with status code", response.status_code)
            print(response.text)
        return response.json()['content'][0]['text']

In [83]:
claude = Claude('Please help me.')
print(claude.chat(ifp.title))

It's difficult to predict with certainty who will win the 2024 Grand Chess Tour, as chess tournaments can be unpredictable and player performance can vary. However, I can provide some context:

1. The players you've mentioned are indeed strong contenders, but they're not the only top players in the world.

2. Other potential winners could include:
   - Magnus Carlsen (if he participates)
   - Ding Liren
   - Ian Nepomniachtchi
   - Hikaru Nakamura
   - Wesley So
   - Levon Aronian
   - Anish Giri

3. The Grand Chess Tour typically features a mix of established top players and rising stars, so there's always potential for surprises.

4. Form and preparation leading up to the tournament will play a crucial role.

5. The tournament format and specific participants for 2024 haven't been announced yet, which could impact the outcome.

Given these factors, it's entirely possible that someone other than the four players you mentioned could win the 2024 Grand Chess Tour. However, without more 

### Perplexity

In [21]:
class Perplexity(LLM):
    def message(self):
        url = "https://api.perplexity.ai/chat/completions"
        headers = {
            "accept": "application/json",
            "authorization": f"Bearer {config.PERPLEXITY_API_KEY}",
            "content-type": "application/json"  }
        payload = {"model": config.PERPLEXITY_MODEL, "messages": self.messages }
        response = requests.post(url=url, json=payload, headers=headers)
        response.raise_for_status()
        return response.json()["choices"][0]["message"]["content"]

In [22]:
perp = Perplexity('You are the most deeply knowledgeable web search engine and insightful analyst')
print(perp.chat(ifp.title))

The question of whether someone other than Fabiano Caruana, Alireza Firouzja, Praggnanandhaa Rameshbabu, or Gukesh Dommaraju will win the 2024 Grand Chess Tour is a topic of ongoing debate and speculation. Here are some key points from the sources:

1. **Field of the 2024 Grand Chess Tour**: The field includes top players such as Fabiano Caruana, Maxime Vachier-Lagrave, Wesley So, Gukesh Dommaraju, Ian Nepomniachtchi, Alireza Firouzja, Anish Giri, Rameshbabu Praggnanandhaa, and Nodirbek Abdusattorov.

2. **Recent Performances**: Alireza Firouzja has shown resilience by bouncing back from a loss in the opening round of the Superbet Chess Classic Romania to win against Nodirbek Abdusattorov. Fabiano Caruana also had a strong game against Wesley So, although it ended in a draw.

3. **Candidates Tournament Predictions**: Fabiano Caruana is considered the favorite to win the 2024 Candidates Tournament due to his consistent performance and extensive experience in such tournaments. However, o

### Metaculus OpenAI

In [84]:
import json

class MetacGPT(LLM):

    def __init__(self, system_role):
        super().__init__(system_role)
        
    def message(self):
        url = "https://www.metaculus.com/proxy/openai/v1/chat/completions"
        headers =  {"Authorization": f"Token {config.METACULUS_TOKEN}",
                    "content-type": "application/json"}
        data = {"model": "gpt-3.5-turbo",
                "max_tokens": 1024,
                "messages": self.messages}
        response = requests.post(url, headers=headers, data = json.dumps(data))
        # Check the response status code
        if response.status_code != 200:
            print("Request failed with status code", response.status_code)
            print(response.text)
        return response.json()['content'][0]['text']

In [85]:
if unit_test and False:
    mgpt = MetacGPT('Please help me')
    mgpt.chat('Bruce Bueno de Mesquita')

### OpenAI

In [25]:
class ChatGPT(LLM):
    def __init__(self, system_role):
        super().__init__(system_role)
        self.client = OpenAI(api_key=config.OPENAI_API_KEY)

    def message(self):
        chat_completion = self.client.chat.completions.create(
            model=config.OPENAI_MODEL,
            messages= self.messages)
        return chat_completion.choices[0].message.content

In [26]:
if unit_test:
    oai = ChatGPT('You are the most deeply knowledgeable web search engine and insightful analyst')
    print(oai.chat(ifp.title))

Predicting the outcome of chess tournaments, including the Grand Chess Tour (GCT), involves a high degree of uncertainty due to the many variables at play. These include player form, physical and mental condition, preparation, and even elements of luck. While Fabiano Caruana, Alireza Firouzja, Praggnanandhaa Rameshbabu, and Gukesh Dommaraju are among the top talents and are considered strong contenders, several other players could also potentially win the 2024 Grand Chess Tour. 

A few notable contenders who could challenge the aforementioned players include:

1. **Magnus Carlsen**: The reigning World Chess Champion (as of my knowledge cut-off in 2021) and often considered one of the greatest chess players of all time. His participation alone raises the competitive bar significantly.

2. **Levon Aronian**: An established Grandmaster who has been a consistent performer in elite tournaments over the years.

3. **Ian Nepomniachtchi**: Known for his sharp and aggressive play, Nepomniachtch

## Agents

#### Agent

In [27]:
class Agent:
    def __init__(self, system_role, llm):
        self.llm = llm(system_role)

    def chat(self, prompt):
        return self.llm.chat(prompt)

#### Question correlator for common topic inference

In [28]:
class QuestionRelator(Agent):
    def __init__(self, llm):
        self.system_role = f"""
You are prompted with list of forecasting questions, each with an id and a title.
Label each question with an underlying event.
If the questions are for the same event, use the same name for the underlying event.
Please return as separate lines formatted as |event|id|title|, do not add any other formatting.
"""
        super().__init__(self.system_role, llm)

    def relate(self, ifps):
        prompt = '\n'.join([f"{ifp.question_id}: {ifp.title}" for ifp in ifps.values()])
        KL = self.chat(prompt)
        K1 = [x.split('|') for x in KL.split('\n')]
        K2 = [(int(id),event) for _,event,id,_,_ in K1]
        for id,event in K2:
            ifps[id].event = event
            print(id, event)

In [29]:
if unit_test:
    qr = QuestionRelator(ChatGPT)
    qr.relate(ifps)

26775 2024 Grand Chess Tour


#### Model domain classifier

In [30]:
class ModelDomainClassifier(Agent):
    def __init__(self, llm):
        self.system_role = f"""
A question about an event is formatted as |id|question|criteria|background|fineprint|.
You will report the model domain of the question, one of

Rate over time
Election outcome
Sports performance
Military conflict
Civil unrest
Dated product announcement
Market price
Macroeconomics
Epidemic
Disease
Drug discovery
Medical device
Crime
Leadership change
Other
"""
        super().__init__(self.system_role, llm)

    def classify(self, ifp):
        prompt = f"|{ifp.question_id}|{ifp.title}|{ifp.resolution_criteria}|{ifp.background}|{ifp.fine_print}|"
        self.R = self.chat(prompt)
        ifp.model_domain = self.R
        print(ifp.question_id, ifp.title)
        print(self.R)

In [31]:
if unit_test:
    mdc = ModelDomainClassifier(ChatGPT)
    mdc.classify(ifp)

26775 Will someone other than Fabiano Caruana, Alireza Firouzja, Praggnanandhaa Rameshbabu, or Gukesh Dommaraju win the 2024 Grand Chess Tour?
Sports performance


#### Rate Analyzer

In [32]:
class RateAnalyzer(Agent):
    def __init__(self, llm):
        self.system_role = f"""
A question about an event is formatted as |id|question|criteria|background|fineprint|.
Today's date is {today}.
If the question is about a level that changes at some rate over time, you will report 
1. today's date, 
2. the end date of the question, 
3. the time in days D from today to end date, 
4. the daily rate of change R of the quantity,
5. today's value V of the quantity,
6. the change in value dV of the quantity = D * r,
7. the final value of the quantity F = V + dV.  Give your best estimate.
Otherwise if the question is about the date that an event will occur, you will report
1. today's date, 
2. the end (measurement) date of the question, 
3. the time in days D from today to end date, 
4. the specific date at which the event is most likely to occur, without reference to the end date of the question
5. whether the likely event date is before, or on or after the question end date
"""
        super().__init__(self.system_role, llm)

    def assess(self, ifp):
        prompt = f"|{ifp.question_id}|{ifp.title}|{ifp.resolution_criteria}|{ifp.background}|{ifp.fine_print}|"
        self.R = self.chat(prompt)
        ifp.rates = self.R
        print(ifp.question_id, ifp.title)
        print(self.R)

In [33]:
if unit_test:
    rates = RateAnalyzer(HuggingChat)
    rates.assess(ifp)

26775 Will someone other than Fabiano Caruana, Alireza Firouzja, Praggnanandhaa Rameshbabu, or Gukesh Dommaraju win the 2024 Grand Chess Tour?
## Step 1: Review the current standings of the 2024 Grand Chess Tour.
The current standings show Fabiano Caruana leading with 22.25 points, followed by Alireza Firouzja with 17.58 points, and Praggnanandhaa Rameshbabu with 16.25 points.

## Step 2: Consider the remaining tournaments and their potential impact on the standings.
Two tournaments remain: the "2024 SAINT LOUIS RAPID & BLITZ" and the "2024 SINQUEFIELD CUP". The former has quicker time controls, while the latter is in a longer, "Classical" time control. This variety could potentially allow for changes in the standings.

## Step 3: Evaluate the possibility of someone other than the current top players winning the Grand Chess Tour.
Given the current standings, it seems challenging for someone other than Caruana, Firouzja, or Praggnanandhaa to win, as they have a significant lead over the

#### Researcher

In [35]:
class Researcher(Agent):
    def __init__(self, llm):
        self.system_role = f"""
You are an open source intelligence analyst.
You summarize news related to questions about events.
A question about an event is formatted as |id|question|criteria|background|fineprint|.
You will find on the web and report any reliable information you can gather about the question.
Do not make an assessment of probability.
Do not repeat information provided to you already in the prompt."""
        super().__init__(self.system_role, llm)

    def research(self, ifp):
        prompt = f"|{ifp.question_id}|{ifp.title}|{ifp.resolution_criteria}|{ifp.background}|{ifp.fine_print}|"
        self.R = self.chat(prompt)
        ifp.research = self.R
        print(ifp.question_id, ifp.title)
        print(self.R)
        return ifp.research

In [36]:
if unit_test:
    researcher = Researcher(Claude)
    researcher.research(ifp);

26775 Will someone other than Fabiano Caruana, Alireza Firouzja, Praggnanandhaa Rameshbabu, or Gukesh Dommaraju win the 2024 Grand Chess Tour?
Based on the information provided in the question, I'll summarize the latest news and information I can find about the 2024 Grand Chess Tour:

1. Current standings: According to the official Grand Chess Tour website, the standings remain as mentioned in the background, with Fabiano Caruana leading, followed by Alireza Firouzja and Praggnanandhaa Rameshbabu.

2. Upcoming tournaments: The two remaining tournaments in the 2024 Grand Chess Tour are:
   - Saint Louis Rapid & Blitz: Scheduled for August 14-18, 2024
   - Sinquefield Cup: Scheduled for August 21-31, 2024

3. Other participants: While the question mentions four specific players, there are nine Tour players in total. The other five players competing in the Tour are:
   - Hikaru Nakamura
   - Wesley So
   - Maxime Vachier-Lagrave
   - Levon Aronian
   - Ian Nepomniachtchi
   - Nodirbek Abd

#### Superforecaster

In [37]:
class Superforecaster(Agent):
    def __init__(self, llm):
        self.system_role = f"""
You are a superforecaster.  
You assign a probability to questions about events.
Questions are given as separate groups of lines formatted as {IFP.forecast_format}.
Groups are separated by '^^^'.
Questions which are about the same event should be assigned consistent probabilities.
Reply to questions with |ASSESSMENT|id|ZZ|rationale| where ZZ is an integer probability from 1 to 99 and rationale is your reasoning for the forecast.
Separate each question with '^^^'.
Do not add any additional headings or group labels or other formatting.
After your initial forecast you may receive feedback of form |CRITIC|id|feedback|.
Reply to each feedback with |id|ZZ|rationale| where ZZ is an integer probability from 1 to 99 and 
and rationale is a revised assessment which may be adjusted from a prior assessment due the feedback unless the feedback is "I concur".
"""
        super().__init__(self.system_role, llm)

    def forecast(self, ifps):
        prompt = '^^^'.join([ifp.record() for ifp in ifps.values()])
        self.F0 = self.chat(prompt)
        self.F1 = [x.strip().replace('\n', '') for x in self.F0.split('^^^')]
        self.F2 = [x.split('|') for x in self.F1] 
        self.F3 = [[x for x in y if x] for y in self.F2]
        self.F4 = [(int(id),int(forecast),rationale) for _, id, forecast, rationale in self.F3]
        for id, forecast, rationale in self.F4:
            ifps[id].forecast = forecast
            ifps[id].rationale = rationale
            print(id, forecast, rationale)

    def reassess(self, ifp):
        prompt = f"|CRITIC|{ifp.question_id}|{ifp.feedback}|"
        self.R0 = self.chat(prompt)
        id,fcst,rationale = [x for x in self.R0.strip().split('|') if x]
        id = int(id)
        fcst = int(fcst)
        ifps[id].forecast = fcst
        ifps[id].rationale = rationale
        print(id, fcst, rationale)

In [38]:
if unit_test:
    sf = Superforecaster(ChatGPT)
    sf.forecast(ifps)

26775 20 The current standings show Fabiano Caruana leading, followed by Alireza Firouzja, and Praggnanandhaa Rameshbabu. While there are two tournaments remaining, Caruana’s significant lead reduces the likelihood of another player winning the Grand Chess Tour. However, it's not impossible given the high variance in chess tournaments and the presence of strong competitors like Hikaru Nakamura and Wesley So, who could potentially perform exceptionally well in the remaining events.


#### Critic

In [90]:
class Critic (Agent):
    def __init__(self, llm):

        self.system_role = f"""
You a very smart and worldly person reviewing a superforecaster's assignment of probabilities to events.
You will receive an event with probabilities given as |event|id|question|zz|rationale|.
zz is an integer probability from 1 to 99 and rationale is the student's logic for assigning probability of zz.
You will reply with a line |id|feedback| where feedback is "I concur" if you see no problem with the rationale and zz otherwise presents possible problems with the rationale and zz.
"""
        super().__init__(self.system_role, llm)

    def feedback(self, ifp):
        self.fb = 'I prasldfkbeddy.'
        prompt = f"|{ifp.event}|{ifp.question_id}|{ifp.title}|{ifp.forecast}|{ifp.rationale}|"
        self.fb = self.chat(prompt)
        if '|' in self.fb: # LLM obeyed system role
            self.fb1 = self.fb.split('|')[2:]
            self.feedback = ''.join([x.strip() for x in self.fb1 if x])
        else:
            self.feedback = self.fb
        ifp.feedback = self.feedback 
        print(ifp.question_id, ifp.feedback)

In [91]:
if unit_test:
    for llm in [MetaAI, Claude, Perplexity, ChatGPT, HuggingChat]:
        print(llm.__name__)
        critic = Critic(llm)
        critic.feedback(ifp) 
        print()

MetaAI
26775 2024 Grand Chess Tour: Can an Outsider Take the Top Spot?
As the 2024 Grand Chess Tour nears its conclusion, the current standings suggest a strong likelihood of one of the top four players - Fabiano Caruana, Alireza Firouzja, Praggnanandhaa Rameshbabu, or Gukesh Dommaraju - claiming the top prize. However, it's essential to remember that chess is a game of unpredictability, and upsets can occur.
Current Standings:
Fabiano Caruana (significant lead)
Alireza Firouzja
Praggnanandhaa Rameshbabu
Dark Horse Contenders:
Hikaru Nakamura
Wesley So
While Caruana's lead is substantial, the remaining two tournaments offer opportunities for other players to make a comeback. If Nakamura or So can perform exceptionally well in the upcoming events, they might be able to bridge the gap and challenge the top contenders.
Conclusion:
Although it's less likely, an outsider can still win the 2024 Grand Chess Tour. The chess world has witnessed numerous upsets in the past, and the presence of s

#### Summarizer

Add an agent to summarize the back and forth between critic and forecaster into a single cogent rationale that incorporates all that was discussed.

## IFP history

In [97]:
history = { '08JUL24': [25876, 25877, 25875, 25873, 25871, 25878, 25874, 25872],
            '09JUL24': [26006, 25936, 25935, 25934, 25933, 26004, 26005],
            '10JUL24': [25955, 25956, 25957, 25960, 25959, 25954, 25953, 25952, 25958],
            '11JUL24': [26019, 26018, 26017, 26020, 26022, 26021, 26023, 26024],
            '12JUL24': [26095, 26096, 26097, 26098, 26099, 26100, 26101, 26102],
            '15JUL24': [26133, 26134, 26138, 26139, 26140, 26157, 26158, 26159],
            '16JUL24': [26189, 26190, 26191, 26192, 26193, 26194, 26195, 26196],
            '17JUL24': [26210, 26211, 26212, 26213, 26214, 26215, 26216],
            '18JUL24': [26232, 26233, 26234, 26235, 26236],
            '19JUL24': [26302, 26303, 26304, 26305, 26306, 26307],
            '22JUL24': [26387, 26388, 26389, 26390, 26391, 26392],
            '23JUL24': [26404, 26405, 26406, 26407, 26408],
            '24JUL24': [26550, 26551, 26552, 26553, 26554, 26555],
            '25JUL24': [26568, 26569, 26570, 26571, 26572, 26573, 26574, 26575, 26576, 26577],
            '26JUL24': [26638, 26639, 26640, 26641, 26642, 26643, 26644, 26645, 26646],
            '29JUL24': [26665, 26666, 26667, 26668, 26669, 26670, 26671, 26683],
            '30JUL24': [26700, 26701, 26702, 26703, 26704, 26705, 26706],
            '31JUL24': [26816, 26817, 26818, 26819, 26820, 26821, 26844],
            '01AUG24': [26771, 26772, 26773, 26774, 26775, 26776, 26777, 26778, 26779, 26780, 26781],
            '02AUG24': [26837, 26838, 26839, 26840, 26841, 26842]}

## Forecasting process

In [122]:
class Forecaster:
    
    def __init__(self, ifps):
        self.max_tries = 4
        self.qr = QuestionRelator(ChatGPT)
        self.ask = AskNews()
        self.researcher = Researcher(Claude)
        self.rates = RateAnalyzer(HuggingChat)
        self.sf = Superforecaster(ChatGPT)
        self.critic = Critic(HuggingChat)
        self.ifps = ifps

    def research(self, ifp):
        self.ask.research(ifp)
        print()
        print('Research for', ifp.question_id)
        self.researcher.research(ifp)
        print()
        print('Rates assessment for', ifp.question_id)
        self.rates.assess(ifp)
        print()
        
    def fit(self):
        print("RELATE QUESTIONS")
        self.qr.relate(self.ifps)
        print()
        print("RESEARCH QUESTIONS")
        for ifp in self.ifps.values():
            print("RESEARCH", ifp.question_id)
            self.research(ifp)
            print()
        print()
        print("SOCRATIC DIALOGUE")
        for ifp in self.ifps.values():
            print("Refining", ifp.question_id)
            for i in range(self.max_tries):
                print("Pass", i, "of", self.max_tries, "on", ifp.question_id)
                if 'concur' in ifp.feedback:
                    print("concur 1")
                    break
                self.critic.feedback(ifp)
                if 'concur' in ifp.feedback:
                    print("concur 2")
                    break
                sf.reassess(ifp)
            print("===============================================")

    def report(self):
        for ifp in self.ifps.values():
            print(ifp.question_id, ifp.title)
            print("Forecast", ifp.forecast)
            print("Rationale", ifp.rationale, '\n')

    def upload(self):
        for ifp in self.ifps.values():
            ifp.upload()

In [110]:
if unit_test:
    test_day = '01AUG24'
    ifps = {id: IFP(id) for id in history[test_day]}

In [123]:
fcst = Forecaster(ifps)

In [ ]:
fcst.fit()

RELATE QUESTIONS
26771 2024 Grand Chess Tour
26772 2024 Grand Chess Tour
26773 2024 Grand Chess Tour
26774 2024 Grand Chess Tour
26775 2024 Grand Chess Tour
26776 Seattle-Tacoma-Bellevue WA Air Quality in Q3 2024
26777 Seattle-Tacoma-Bellevue WA Air Quality in Q3 2024
26778 Seattle-Tacoma-Bellevue WA Air Quality in Q3 2024
26779 Seattle-Tacoma-Bellevue WA Air Quality in Q3 2024
26780 Bitcoin New All-Time High by October 1, 2024
26781 William Ruto's Presidency

RESEARCH QUESTIONS
RESEARCH 26771
News for 26771
World Chess Champion Magnus Carlsen lost to Richard Rapport of Hungary in the 4th round of the World Rapid and Blitz Chess Team Championship. Carlsen, playing for the WR Chess Team, lost in 23 moves with white pieces. His teammate, Russian grandmaster Ian Nepomniachtchi, drew with Hikaru Nakamura's opponent, Aronian's teammate, in his match. As a result, WR Chess Team lost 1-5 to Chessy. After five rounds, Al-Ain ACMG UAE is leading with 21.5 points, with Chessy trailing by 0.5 poi

In [ ]:
fcst.report()

## Daily forecast

### Get IFP ids

In [ ]:
ifps = list_questions()['results']
today_ids = list(sorted([x['id'] for x in ifps]))
print(today, today_ids)

## Forecast

In [ ]:
ifps = {id: IFP(id) for id in today_ids}

In [ ]:
fcst = Forecaster(ifps)
fcst.fit()

In [ ]:
fcst.report()

In [ ]:
fcst.upload()